In [105]:
import pandas as pd, numpy as np
from influxdb import DataFrameClient
import json
%cd /home/jovyan/work

/home/jovyan/work


Set params

In [106]:
purge=False
# purge=True

In [107]:
process_annoations=False
# process_annoations=True

In [108]:
# process_macro=False
process_macro=True

In [109]:
# write_out=False
write_out=True

In [110]:
full_overwrite=False
# full_overwrite=True

Load pre-params

In [111]:
df0=pd.read_csv('df0.csv').set_index('date')
df0.index=pd.to_datetime(df0.index)

In [112]:
bets=json.loads(open('bets.json','r').read())

In [113]:
severity=pd.read_csv('severity.csv').set_index('date')
severity.index=pd.to_datetime(severity.index)

In [114]:
mobility=pd.read_csv('mobility.csv').set_index('date')
mobility.index=pd.to_datetime(mobility.index)

In [115]:
mobility_mini=pd.read_csv('mobility_mini.csv').set_index('date')
mobility_mini.index=pd.to_datetime(mobility_mini.index)

Establish DB connection

In [116]:
user = 'admin'
password = open('auth/influxa.txt','r').read()
host='influxdb'
port=8086
dbname='base'
dbname_long='long'
protocol = 'line' #'json'
client = DataFrameClient(host, port, user, password, dbname)
client_long = DataFrameClient(host, port, user, password, dbname_long)

In [117]:
if purge:
    client.drop_database(dbname)
    client.drop_retention_policy(dbname)
    client.create_database(dbname)
    client.create_retention_policy(dbname, '600d', 1, default=True)
    client_long.drop_database(dbname_long)
    client_long.drop_retention_policy(dbname_long)
    client_long.create_database(dbname_long)
    client_long.create_retention_policy(dbname_long, '6000d', 1, default=True)

In [118]:
htmlipath='../html/'
htmlepath='//myserv-html.er/'
htmlepath_other='//mybackupserv-html.er/'

In [119]:
titles={'HU':"Magyar",'RO':'Română','EN':'English'}

In [120]:
rtitles={titles[t]:t for t in titles}

In [121]:
import requests
grafana = "http://grafana:3000/"
headers = {
    'Authorization': 'Bearer '+open('auth/grafana.txt','r').read(),
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}

In [122]:
response = requests.get(grafana+'api/folders', headers=headers)
folders=json.loads(response.content)

In [123]:
folder_id=[f['id'] for f in folders if f['title']=='My Grafana Folder'][0] #General

In [124]:
response = requests.get(grafana+'api/search?folderIds='+str(folder_id), headers=headers)
dashs=json.loads(response.content)

In [125]:
uids={rtitles[d['title']]:d['uid'] for d in dashs if d['title'] in rtitles}
iids={rtitles[d['title']]:d['id'] for d in dashs if d['title'] in rtitles}

In [126]:
uids_light={rtitles[d['title'].split(' Light')[0]]:d['uid'] for d in dashs if 'Light' in d['title']}
iids_light={rtitles[d['title'].split(' Light')[0]]:d['id'] for d in dashs if 'Light' in d['title']}

In [127]:
# response = requests.get(grafana+'api/dashboards/uid/hu-test', headers=headers)
# model=json.loads(response.content)['dashboard']
# open('hu-test.json','w').write(json.dumps(model))

In [128]:
languages=['HU','RO','EN']

In [129]:
url='https://docs.google.com/spreadsheets/d/'+open('auth/sheet.txt','r').read()+'/gviz/tq?tqx=out:csv&sheet='

In [130]:
def get_szotar():
    sheet='szotar'
    columns=languages+[i+'_description' for i in languages]+[i+'_source' for i in languages]
    df=pd.read_csv(url+sheet)
    df=df[['ID','UI']+columns]
    sheet='minidashboard'
    df2=pd.read_csv(url+sheet)
    df2=df2[['ID','UI']+columns]
    df=pd.concat([df,df2])
    szotardf=df.set_index('ID')[columns]
    szotar=df.set_index('ID').T.to_dict()
    szotarHU=df.set_index('HU',drop=False).T.to_dict()
    szotarRO=df.set_index('RO',drop=False).T.to_dict()
    szotarEN=df.set_index('EN',drop=False).T.to_dict()
    return szotardf,szotar,szotarHU,szotarRO,szotarEN

In [131]:
def resolve_time_conflicts(series):
    ds={}
    ts=[]
    for d in series:
        if d not in ds:
            ds[d]=pd.to_datetime(d)
            t=(ds[d])
        else:
            ds[d]=ds[d]+pd.to_timedelta('193m')
            t=(ds[d])
        ts.append(t)
    return ts

# Influx

In [132]:
szotardf,szotar,szotarHU,szotarRO,szotarEN=get_szotar()

<ipython-input-130-77652dbc15c9>:12: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarHU=df.set_index('HU',drop=False).T.to_dict()
<ipython-input-130-77652dbc15c9>:13: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarRO=df.set_index('RO',drop=False).T.to_dict()
<ipython-input-130-77652dbc15c9>:14: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarEN=df.set_index('EN',drop=False).T.to_dict()


In [133]:
import time
import datetime
import pytz
utc=pytz.UTC

In [134]:
latest=df0.index[-1].date()
now=str(pd.to_datetime("now"))[:10]
print(latest,now)

2021-05-02 2021-05-02


In [135]:
current=int(df0['case14'][-1])

# Grafana

In [136]:
today0=now[-2:]
if today0[0]=='0': 
    today=today0[1]
else:
    today=today0
currents=str(current)[:-3]+'&nbsp;'+str(current)[-3:]
hour=str(pd.to_datetime('now')+pd.to_timedelta('3h'))[11:16]
str(szotar['report']['UI'][1:]),current,currents,today0,hour

('432', 28351, '28&nbsp;351', '02', '01:41')

In [137]:
def darken2(color):
    return '#'+str(hex(int("0x"+color[1:], 16) & 0xfefefe >> 1))[2:]
def brighten2(color):
    return '#'+str(hex(int("0x"+color[1:], 16) & 0x7f7f7f << 1))[2:]
def adjust_lightness(color, amount=0.5):
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return mc.rgb2hex(colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2]))

In [138]:
# szotardf,szotar,szotarHU,szotarRO,szotarEN=get_szotar()

In [139]:
colors_to_darken=['#F2CC0C','#CA95E5','#FF780A','#E0B400','#96D98D','#F2495C','#E02F44',
                  'lime','#73BF69','#8AB8FF','#3274D9','#D3D3D3','#FF7383','#FADE2A']

# Maps

In [140]:
top=3

In [141]:
iso_counties=json.loads(open(htmlipath+'panels/iso_counties.json','r').read())

In [142]:
georo=json.loads(open(htmlipath+'panels/romania-counties.json','r').read())
georoco={i['properties']['NAME_1']:i['properties']['ID_1'] for i in georo['objects']['ROU_adm1']['geometries']}   
georoco['București']=georoco['Bucharest']

Locality map

In [143]:
datagov2a=pd.read_excel('https://data.gov.ro/dataset/b86a78a3-7f88-4b53-a94f-015082592466/resource/d0b60b45-fb08-4980-a34c-8cbb4a43cad3/download/transparenta_martie_2021.xlsx',skiprows=1)
datagov2b=pd.read_excel('https://data.gov.ro/dataset/b86a78a3-7f88-4b53-a94f-015082592466/resource/d3280256-07cc-4f93-957a-9815085899be/download/transparenta_aprilie_2021.xlsx',skiprows=1)
datagov2c=pd.read_excel('https://data.gov.ro/dataset/b86a78a3-7f88-4b53-a94f-015082592466/resource/12980ac2-f459-4b42-905d-16b93bde62ba/download/transparenta_mai_2021.xlsx',skiprows=1)

In [144]:
datagov2=datagov2a.set_index(['Judet','UAT']).join(datagov2b.set_index(['Judet','UAT'])).join(datagov2c.set_index(['Judet','UAT'])).reset_index()

In [145]:
# del datagov1a
# del datagov1b
del datagov2a
del datagov2b
del datagov2c

In [146]:
uat=json.loads(open(htmlipath+'panels/uat_simplificat.geojson','r').read())

In [147]:
locs=[]
for f in uat['features']:
    locs.append(f['properties'])
locs=pd.DataFrame(locs)[['judet','uat','siruta']]

In [148]:
def judetconverter(c):
    return (' '.join([i.capitalize() for i in c.split(' ')])).replace('ţ','ț').replace('ş','ș').replace('năsăud','Năsăud').\
        replace('severin','Severin').replace('mare','Mare').\
        replace('Municipiul București','Bucharest').replace('București','Bucharest')

In [149]:
geo_iso=pd.DataFrame(iso_counties,index=['index']).T.reset_index().set_index('index').join(\
    pd.DataFrame(georoco,index=['geo_id']).T.reset_index().set_index('index')).reset_index()
geo_iso.columns=['en','iso2','geo_id']

In [150]:
locs['judet_norm']=[judetconverter(i) for i in locs['judet']]
datagov2['judet_norm']=[judetconverter(i) for i in datagov2['Judet']]

In [151]:
def uatconverter(judet,c):
    c=(' '.join(['-'.join([b.capitalize() for b in a.split('-')]) for a in c.split(' ')]))\
        .replace('ţ','ț').replace('ş','ș').replace('Ţ','Ț').replace('Ş','Ș')\
        .replace('î','â').replace(' De ',' de ')\
        .replace('Cuza Voda','Cuza Vodă')
    
    if (judet=='Dolj'):
        c=c.replace('Segarcea','Șegarcea')
    elif (judet=='Teleorman'):
        c=c.replace('Turnu Magurele','Turnu Măgurele')
    elif (judet=='Olt'):
        c=c.replace('Ipotesti','Ipotești')
    elif (judet=='Alba'):
        c=c.replace('Râmetea','Rimetea')
    elif (judet=='Constanța'):
        c=c.replace('44066','')
    elif (judet=='Ialomița'):
        c=c.replace('Radulești','Rădulești')
    elif (judet=='Argeș'):
        c=c.replace('Ciofringeni','Ciofrângeni')\
        .replace('Valea Mare Pravăț','Valea Mare-Pravăț')
    elif (judet=='Vâlcea'):
        c=c.replace('Păusești-Măglași','Păușești-Măglași')
    elif (judet=='Prahova'):
        c=c.replace('Cocorastii Colt','Cocorăștii Colț')
    elif (judet=='Bucharest'):
        c=c.replace('București Sectorul','Sector')
    elif (judet=='Gorj'):
        c=c.replace('Crușet','Crușeț')
    elif (judet=='Brăila'):
        c=c.replace('Racoviță','Racovița')\
        .replace('Gradiștea','Grădiștea')\
        .replace('Chișcani','Chiscani')
    elif (judet=='Cluj'):
        c=c.replace('Râșca','Rișca')
    elif (judet=='Botoșani'):
        c=c.replace('Răușeni','Răuseni')
    elif (judet=='Maramureș'):
        c=c.replace('Șisești','Șișești')
    elif (judet=='Brașov'):
        c=c.replace('Sambata de Sus','Sâmbăta de Sus')
    elif (judet=='Vaslui'):
        c=c.replace('Tătărăni','Tătărani')
    elif (judet=='Iași'):
        c=c.replace('Țigănăși','Țigănași')
    elif (judet=='Bistrița-Năsăud'):
        c=c.replace('Ilva Mica','Ilva Mică')
    elif (judet=='Bihor'):
        c=c.replace('Sănnicolau Romăn','Sânnicolau Român')
    elif (judet=='Mureș'):
        c=c.replace('Sarmașu','Sărmașu')
    return c

In [152]:
datagov2['uat_norm']=[(' '.join(['-'.join([b.capitalize() for b in a.split('-')]) for a in u.split(' ')]))\
                      .replace('ţ','ț').replace('ş','ș').replace('Ţ','Ț').replace('Ş','Ș')\
                      .replace(' De ',' de ').replace('Municipiul ','').replace('Oraș ','')\
                      for u in datagov2['UAT']]

In [153]:
uat_norms=[]
for judet in locs['judet_norm'].unique():
    for l in locs[locs['judet_norm']==judet]['uat'].unique():
        u=uatconverter(judet,str(l))
        if u not in datagov2[datagov2['judet_norm']==judet]['uat_norm'].unique():
            u=u.replace(' ','-')
            if u not in datagov2[datagov2['judet_norm']==judet]['uat_norm'].unique():
                print(judet,l,u)
                u=''
        uat_norms.append({'judet_norm':judet,'uat_norm':u,'uat':l})

Constanța 44066 
  


In [154]:
locs2=locs.set_index(['judet_norm','uat']).join(pd.DataFrame(uat_norms).set_index(['judet_norm','uat'])).reset_index()

In [155]:
datagov3=datagov2.set_index(['judet_norm','uat_norm']).join(locs2.set_index(['judet_norm','uat_norm']))
datagov3=datagov3.set_index(['judet','uat','siruta'],append=True).drop(['UAT','Judet'],axis=1)
datagov3.index=datagov3.index.reorder_levels([2,3,4,0,1])
datagov3_dates=[str(pd.to_datetime(c)) for c in datagov3.columns]

In [156]:
datagov3=datagov3.round(2)

In [157]:
datagov3.reset_index().to_csv('github/data/time_series_ro_uat_daily.csv')

In [158]:
del datagov2

In [ ]:
def cat(r):
    if r<1: return '< 1 ‰'
    elif r<2: return '1-2 ‰'
    elif r<4: return '2-4 ‰'
    elif r<6: return '4-6 ‰'
    elif r<8: return '6-8 ‰'
    elif r<10: return '8-10 ‰'
    elif r<20: return '> 10-20 ‰'
    else: return '> 20 ‰'
def cat2(r):
    if r<1: return 0
    elif r<2: return 1
    elif r<4: return 2
    elif r<6: return 3
    elif r<8: return 4
    elif r<10: return 5
    elif r<20: return 6
    else: return 7

In [ ]:
t=0
new_uat={'type':'FeatureCollection','features':[]}
new_uat_numbers=[]
new_uat_daily={'type':'FeatureCollection','features':[]}
for i in range(len(uat['features'])):
    dummy={}
    judet=uat['features'][i]['properties']['judet']
    if (judet!=''):
        u=uat['features'][i]['properties']['uat']
        siruta=uat['features'][i]['properties']['siruta']
        # dummy={'judet':judet,'uat':u,'siruta':siruta}
        if u in datagov3.loc[judet].index:
            d=datagov3.loc[judet].loc[u].loc[siruta]
            values=d.values[0]
            dummy['🌄']=d.index[0][0]
            dummy['🏠']=d.index[0][1]
            for t in range(len(values)):
                value=values[t]
                date=datagov3_dates[t]
                dummy['📈']=value
                dummy['🔴']=cat(value)
                dummy['⚫']=cat2(value)
                dummy['📆']=date
                feature={'type':'Feature','geometry':uat['features'][i]['geometry'],'properties':dummy.copy()}
                new_uat['features'].append(feature)
                new_uat_numbers.append(dummy.copy())
                if (t==len(datagov3_dates)-1):
                    new_uat_daily['features'].append(feature)

In [ ]:
open(htmlipath+'panels/new_uat_numbers.json','w').write(json.dumps(new_uat_numbers))
# open(htmlipath+'panels/new_uat.json','w').write(json.dumps(new_uat))

In [ ]:
#Kepler.gl export
open(htmlipath+'panels/daily/new_uat'+date[:10]+'.json','w').write(json.dumps(new_uat_daily))
open(htmlipath+'panels/new_uat_daily.json','w').write(json.dumps(new_uat_daily))

In [ ]:
del new_uat

In [ ]:
#D3plus export

In [ ]:
datagov4=datagov3.stack().reset_index()
datagov4.columns=['judet','uat','id','judet_norm','uat_norm','date','value']
datagov4['value']=np.round(datagov4['value'],2)
# datagov4=datagov4.dropna(subset=['judet'])
datagov4=datagov4.dropna(how='any')

In [ ]:
open(htmlipath+'panels/daily/uat2b_'+date[:10]+'.json','w').write(json.dumps(list(datagov4.T.to_dict().values())))

In [ ]:
# #HTML overwrite
# cases_localities=open(htmlipath+'panels/cases_localities.html','r').read()
# cases_localities=cases_localities[:cases_localities.find('daily/uat2b_\' + \'2021-')+17]+latest+cases_localities[cases_localities.find('daily/uat2b_\' + \'2021-')+27:]
# open(htmlipath+'panels/cases_localities.html','w').write(cases_localities)
# #### CHECK IF LATEST DATE IS ONE LESS than latest

Restrictions

In [ ]:
df2=pd.DataFrame(new_uat_numbers)
df4=df2[['🌄', '🏠', '📈','📆']].set_index(['🌄', '🏠', '📆']).unstack()['📈'].sort_index(axis=1)

In [ ]:
def restrictie(x,w):
    if x>7.5: return 4
    elif x>7:
        if w>7.5: return 3
        else: return 2
    elif x>4: return 2
    elif x>3.5:
        if w>4: return 1
        else: return 0
    else:
        return 0
    
def restrictie2(x,w):
    if x>7.5: return '7.5+'
    elif x>7:
        if w>7.5: return '7.5-'
        else: return '4+'
    elif x>4: return '4+'
    elif x>3.5:
        if w>4: return '4-'
        else: return '0'
    else:
        return '0'

In [ ]:
dr=[]
for i in df4.T.iteritems():
    v=i[1]
    #print(i[0])
    for j in range(len(v)):
        t=v.index[j]
        x=v[j]
        y=v[j-min(14,j):j].values
        w=x
        if len(y):
            w=max(y)
        r=restrictie(x,w)
        #print(j,t,x,w,r)
        dr.append({'🌄':i[0][0], '🏠':i[0][1], '🚦':r,'📆':t})

In [81]:
dp=pd.DataFrame(dr).set_index(['🌄', '🏠','📆']).astype(float)

In [82]:
for i in new_uat_daily['features']:
    i['properties']['🚦']=dp.loc[i['properties']['🌄']].loc[i['properties']['🏠']].loc[i['properties']['📆']]['🚦']

In [ ]:
open(htmlipath+'panels/new_uat_daily2.json','w').write(json.dumps(new_uat_daily))
open(htmlipath+'panels/daily/new_uat_daily2_'+date[:10]+'.json','w').write(json.dumps(new_uat_daily))
open('uat_daily.json','w').write(json.dumps(new_uat_daily))

In [85]:
#HTML overwrite
uat_html=open(htmlipath+'panels/daily/mapbox-'+date[:10]+'.html','r').read()
uat_html=uat_html.replace(date[5:10],date[5:8]+today0).replace(date[5:10],date[5:8]+today0)
uat_html=uat_html[:uat_html.find('.json')-2]+\
    str(pd.to_datetime(now)+pd.to_timedelta('-1d'))[8:10]+\
    uat_html[uat_html.find('.json'):]
open(htmlipath+'panels/daily/mapbox-'+date[:8]+today0+'.html','w').write(uat_html)

21104

Upload to `uat.json` to mapbox

In [88]:
#Mapbox colors:
# 239e69
# f5d232
# c52b69
# 850237

In [89]:
#Add szekelydata GitHub update
#Add mapbox direct upload

In [90]:
#Sanity check
datagov3.loc['Cluj'].loc['Cluj-Napoca'].T.tail(3)

siruta,54975
judet_norm,Cluj
uat_norm,Cluj-Napoca
2021-04-29,2.62
2021-04-30,2.49
2021-05-01,2.33


Mapbox upload

In [91]:
import requests

In [92]:
mapbox_api_key = open('auth/mapbox.txt','r').read()

Create tileset source

In [93]:
!tilesets validate-source uat_daily.json

Validating features
✔ valid


In [ ]:
!tilesets upload-source <MAPBOX_USER> roeim uat_daily.json --replace --token $mapbox_api_key

In [95]:
%%capture cap_out
!tilesets publish <MAPBOX_USER>.roeim --token $mapbox_api_key

In [ ]:
jobid=json.loads(cap_out.stdout.split('}')[0]+'}')['jobId']
!tilesets job <MAPBOX_USER>.roeim $jobid --token $mapbox_api_key

Wait 3 min

In [ ]:
import time
for i in range(18):
    time.sleep(10)
    print(i)

Push to Grafana

In [150]:
nevuto={'0':'é','1':'é','2':'á','3':'á','4':'é','5':'é','6':'á','7':'é','8':'á','9':'é'}
if len(today)<2: space=' '
else: space=''
for lang in languages:
# for lang in ['HU']:
    response = requests.get(grafana+'api/dashboards/uid/'+lang.lower(), headers=headers)
    model=json.loads(response.content)['dashboard']
    for i in model['panels']:
        if lang=='HU':
            #print(i['id'],i['type'],i['title'])
            if i['id'] in (179, 91):
                i['options']['iframeURL']=i['options']['iframeURL'][:i['options']['iframeURL'].find('.html#')-2]+\
                    today0+i['options']['iframeURL'][i['options']['iframeURL'].find('.html#'):]
                i['description']=i['description'][:i['description'].find('n,')-4].strip()+' '+today+'-'+\
                    nevuto[now[-1]]+i['description'][i['description'].find('n,'):\
                    i['description'].find(':')-2]+hour+\
                    i['description'][i['description'].find(':')+3:]
                i['title']=i['title'][:i['title'].rfind(' ')]+' '+today
            if i['id']==82:
                i['content']=i['content'][:i['content'].find('<b>')+3]+str(szotar['report']['UI'][1:])+\
                    i['content'][i['content'].find('</b>'):i['content'].rfind('<b>')+3]+currents+\
                    i['content'][i['content'].rfind('</b>'):]
            if i['id']==93:
                i['content']=i['content'][:i['content'].rfind('</b>')-2]+today0+\
                i['content'][i['content'].rfind('</b>'):]
        elif lang=='EN':
            if i['id'] in (153, 91):
                i['options']['iframeURL']=i['options']['iframeURL'][:i['options']['iframeURL'].find('.html#')-2]+\
                    today0+i['options']['iframeURL'][i['options']['iframeURL'].find('.html#'):]
                i['description']=i['description'][:i['description'].find(':')-2]+hour+' on '+today+' '+ i['description'][i['description'].find('on ')+5:]
                i['title']=i['title'][:i['title'].find('📆')+2]+today+i['title'][i['title'].find('📆')+3:]
            if i['id']==82:
                i['content']=i['content'][:i['content'].find('<b>')+3]+str(szotar['report']['UI'][1:])+\
                    i['content'][i['content'].find('</b>'):i['content'].rfind('<b>')+3]+currents+\
                    i['content'][i['content'].rfind('</b>'):]
            if i['id']==93:
                i['content']=i['content'][:i['content'].rfind('</b>')-2]+today0+\
                i['content'][i['content'].rfind('</b>'):]
        elif lang=='RO':
            if i['id'] in (153, 91):
                i['options']['iframeURL']=i['options']['iframeURL'][:i['options']['iframeURL'].find('.html#')-2]+\
                    today0+i['options']['iframeURL'][i['options']['iframeURL'].find('.html#'):]
                i['description']=i['description'][:i['description'].find(':')-2]+hour+' în data de '+today+ i['description'][i['description'].find('în data de ')+12:]
                i['title']=i['title'][:i['title'].find('📆')+2]+today+i['title'][i['title'].find('📆')+3:]
            if i['id']==82:
                i['content']=i['content'][:i['content'].find('<b>')+3]+str(szotar['report']['UI'][1:])+\
                    i['content'][i['content'].find('</b>'):i['content'].rfind('<b>')+3]+currents+\
                    i['content'][i['content'].rfind('</b>'):]
            if i['id']==93:
                i['content']=i['content'][:i['content'].rfind('<b>')+3]+today0+\
                i['content'][i['content'].rfind('<b>')+5:]
            
    r=requests.post(grafana+'api/dashboards/db', headers=headers, json={"dashboard":model,
                                                                            "folderId": folder_id,
                                                                            "overwrite": True
                                                                           })       
    print(r,r.content)
    open(lang+'.json','w').write(json.dumps(model))    
    
    model_light=json.dumps(model).replace('dark','light',9999).replace('lightGray','#52545C',9999)\
        .replace('#d3d3d3','#52545c',9999)\
        .replace('csaladen.es/favicon.ico" style="','csaladen.es/favicon.ico" style="filter: brightness(0.3);',9999)
    for color in colors_to_darken:
        model_light=model_light.replace(color,adjust_lightness(color,0.8),9999)
        model_light=model_light.replace(color.lower(),adjust_lightness(color,0.8),9999)
    model=json.loads(model_light)
    model['title']=titles[lang]+' Light'
    model['uid']=uids_light[lang]
    model['id']=iids_light[lang]
    r=requests.post(grafana+'api/dashboards/db', headers=headers, json={"dashboard":model,
                                                                            "folderId": folder_id,
                                                                            "overwrite": True
                                                                           })    
    print(r,r.content)
    open(lang+'-light.json','w').write(json.dumps(model))   

<Response [200]> b'{"id":2,"slug":"magyar","status":"success","uid":"hu","url":"/d/hu/magyar","version":373}'
<Response [200]> b'{"id":5,"slug":"magyar-light","status":"success","uid":"hu-light","url":"/d/hu-light/magyar-light","version":189}'
<Response [200]> b'{"id":3,"slug":"romana","status":"success","uid":"ro","url":"/d/ro/romana","version":283}'
<Response [200]> b'{"id":6,"slug":"romana-light","status":"success","uid":"ro-light","url":"/d/ro-light/romana-light","version":183}'
<Response [200]> b'{"id":4,"slug":"english","status":"success","uid":"en","url":"/d/en/english","version":237}'
<Response [200]> b'{"id":7,"slug":"english-light","status":"success","uid":"en-light","url":"/d/en-light/english-light","version":181}'
